In [ ]:
# optical depth plots
import sys
sys.path.append('../../')
from fig1_calibration import create_sim as cs
import pandas as pd
import numpy as np
import itertools
import sciris as sc
bounds = cs.define_pars(which='bounds',use_safegraph=True)
bounds['tn'][0] = 5.0
keys = list(bounds.keys())
mapping = sc.objdict({
    'beta'  : r'Overall $\beta$',
    'bc_wc1': r'Work/community $\beta$ reduction',
    'bc_lf' : r'LTCF $\beta$ reduction',
    'tn'   : 'Symptomatic testing OR',
    })
bounds


In [ ]:
import matplotlib.pyplot as plt
from itertools import combinations
# plotting
#%config InlineBackend.figure_formats = ['svg']
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Times']})
rc('text', usetex=False)
df = pd.read_csv('../hm_waves/NI_pars_wave003.csv')


def make_opt_depth_plot(df,k1,k2):
    b1 = np.linspace(bounds[k1][0],bounds[k1][1],15)
    b2 =  np.linspace(bounds[k2][0],bounds[k2][1],15)
    H, xedges, yedges = np.histogram2d(df[k1].values, df[k2].values, bins=[b1,b2])
    return H.T

def opt_depth(df,plot_type='matrix'):

    fig, ax = plt.subplots(nrows=2,ncols=3,figsize=(11.5,8.5),layout='compressed')
    ax = ax.flatten()
    keypairs = list(combinations(keys,2))
    
    for i, pair in enumerate(keypairs):
        k1, k2 = pair
        mat  = make_opt_depth_plot(df,k1,k2)
        mat  = mat / mat.sum()
        nticks = 3
        xticks = np.linspace(bounds[k1][0],bounds[k1][1],nticks)
        yticks = np.linspace(bounds[k2][0],bounds[k2][1],nticks)
        if plot_type== 'matrix':
            tmp = ax[i].imshow(mat,origin='lower',extent=[0,1,0,1],cmap='magma',vmin=0.0)
        elif plot_type == 'contour':
            cs = ax[i].contourf(mat,origin=None,extent=[0,1,0,1],cmap='magma',
                                 vmin=0.0,vmax=mat.max(),levels=80,antialiased=True)
            cs.set_edgecolor('face')
            fig.colorbar(cs,ax=ax[i],shrink=0.9,
                         ticks = np.linspace(0,mat.max(),3),format='%.2f')
        ax[i].set_xticks(np.linspace(0,1,nticks))
        ax[i].set_xticklabels(np.round(xticks,3))
        ax[i].set_yticks(np.linspace(0,1,nticks))
        ax[i].set_yticklabels(np.round(yticks,3))
        ax[i].set_ylabel(mapping[k2])
        ax[i].set_xlabel(mapping[k1])
    ax = ax.reshape((2,3))
    return fig, mat, cs, ax
fig, mat, tmp, ax = opt_depth(df,plot_type='contour')

fig.savefig('../figs/optical-depth-w-colorbar.pdf')
